<a href="https://colab.research.google.com/github/yashvivaghela04/FACE-RECOGNITION-DETECTION/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Install required libraries
!pip install deepface opencv-python

import cv2
from deepface import DeepFace
import os

# Paths
group_photo_path = "/content/group photo.jpeg"
cropped_faces_dir = "/content/cropped_faces"
os.makedirs(cropped_faces_dir, exist_ok=True)

# Load Haar cascade for face detection
haar_cascade_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
haar_cascade = cv2.CascadeClassifier(haar_cascade_path)

# Detect and crop faces
group_img = cv2.imread(group_photo_path)
gray_img = cv2.cvtColor(group_img, cv2.COLOR_BGR2GRAY)
faces = haar_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5)

for i, (x, y, w, h) in enumerate(faces):
    cropped_face = group_img[y:y+h, x:x+w]
    cropped_face_path = f"{cropped_faces_dir}/face_{i}.jpg"
    cv2.imwrite(cropped_face_path, cropped_face)
    print(f"Saved cropped face: {cropped_face_path}")

print("Face detection and cropping completed.")


Saved cropped face: /content/cropped_faces/face_0.jpg
Saved cropped face: /content/cropped_faces/face_1.jpg
Saved cropped face: /content/cropped_faces/face_2.jpg
Saved cropped face: /content/cropped_faces/face_3.jpg
Saved cropped face: /content/cropped_faces/face_4.jpg
Saved cropped face: /content/cropped_faces/face_5.jpg
Face detection and cropping completed.


In [9]:
import numpy as np

# Function to generate embeddings
def generate_embeddings(face_images_dir):
    embeddings = {}
    for face_image in os.listdir(face_images_dir):
        face_path = os.path.join(face_images_dir, face_image)
        try:
            embedding = DeepFace.represent(face_path, model_name="Facenet", enforce_detection=False)
            embeddings[face_image] = embedding[0]["embedding"]
            print(f"Generated embedding for {face_image}")
        except Exception as e:
            print(f"Error generating embedding for {face_image}: {e}")
    return embeddings

# Generate and store embeddings
face_embeddings = generate_embeddings(cropped_faces_dir)
print("Embeddings generated for all cropped faces.")


Generated embedding for face_2.jpg
Generated embedding for face_3.jpg
Generated embedding for face_5.jpg
Generated embedding for face_4.jpg
Generated embedding for face_1.jpg
Generated embedding for face_0.jpg
Embeddings generated for all cropped faces.


In [10]:
# Target image path
target_image_path = "/content/SRK.png"

# Function to verify target face against stored embeddings
def verify_face(target_image_path, face_embeddings):
    try:
        # Generate embedding for the target image
        target_embedding = DeepFace.represent(target_image_path, model_name="Facenet", enforce_detection=False)[0]["embedding"]
        print("Generated embedding for target image.")

        # Compare with stored embeddings
        match_found = False
        for face_name, stored_embedding in face_embeddings.items():
            similarity = np.linalg.norm(np.array(target_embedding) - np.array(stored_embedding))
            if similarity < 10:  # Similarity threshold; adjust as needed
                print(f"True:Match found with {face_name}! Similarity score: {similarity:.2f}")
                match_found = True
                break

        if not match_found:
            print("False:No match found.")
    except Exception as e:
        print(f"Error verifying target image: {e}")

# Verify target image
verify_face(target_image_path, face_embeddings)


Generated embedding for target image.
True:Match found with face_3.jpg! Similarity score: 8.13
